In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load your dataset
df = pd.read_csv("books.csv")


In [4]:
# Tokenize and pad sequences for book descriptions
max_words = 5000  # Adjust based on your dataset
max_len = 100  # Adjust based on your dataset

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['description'])
sequences = tokenizer.texts_to_sequences(df['description'])
padded_sequences = pad_sequences(sequences, maxlen=max_len)


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
# Build LSTM model
embedding_dim = 50  # Adjust based on your preferences
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(64, activation='relu'))
model.add(Dense(max_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss=tf.compat.v1.losses.sparse_softmax_cross_entropy, metrics=['accuracy'])


In [ ]:
# Function to get embeddings for user input
def get_user_embedding(user_input):
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    user_embedding = model.predict(padded_sequence)
    return user_embedding.flatten()


In [ ]:
# User input
user_input = "gravity"  # Replace with the actual user input
user_embedding = get_user_embedding(user_input)

# Process book data
book_embeddings = np.array([get_user_embedding(description) for description in df['description']])


In [ ]:
# Calculate cosine similarity between user input and book embeddings
user_embedding = user_embedding.reshape(1, -1)  # Reshape to 2D array
similarities = cosine_similarity(user_embedding, book_embeddings)

# Print top 10 similarities along with corresponding books
print("Top 10 Similarities:")
top_indices = similarities.argsort()[0][-10:][::-1]
for i, idx in enumerate(top_indices):
    similarity_score = similarities[0, idx]
    book_title = df.iloc[idx]['title']
    book_description = df.iloc[idx]['description']
    
    print(f"{i + 1}. Similarity: {similarity_score:.4f} - Title: {book_title}")

# Display top N recommendations
top_recommendations = df.iloc[top_indices]
print(top_recommendations[['title']])
